In [14]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")
Pkg.add("Cpp")
Pkg.add("Cxx")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed
INFO: Package GLPKMathProgInterface is already installed
INFO: Package Cpp is already installed
INFO: Package Cxx is already installed


In [15]:
using JuMP
using MathProgBase
using GLPKMathProgInterface
using Cpp

In [20]:
include("util.jl")
include("network.jl")

In [10]:
# read in neural networks
small_nnet = read_nnet("small_nnet.txt")
large_nnet = read_nnet("ACASXU_nnet_1.txt")

Network(Layer[Layer([0.22763; -0.188762; … ; -0.44755; 0.175917], [0.0540062 0.0540062 … -0.180027 0.242194; -1.12374 -1.12374 … -0.00917929 0.055623; … ; -1.57631 -1.57631 … -0.00832176 -0.0926607; 0.620367 0.620367 … -0.0146141 0.262575]), Layer([0.08355; -0.0213261; … ; -0.426623; 0.780925], [-0.184202 -0.184202 … 0.337388 0.395671; 0.0171639 0.0171639 … -0.0371157 0.0196442; … ; -1.24508 -1.24508 … 0.646698 -0.159838; -0.348713 -0.348713 … -0.60238 -0.142155]), Layer([-0.408486; 1.06875; … ; -0.464336; 0.114195], [0.146062 0.146062 … -1.15657 0.515279; -1.159 -1.159 … -2.33211 -0.517898; … ; -1.14982 -1.14982 … -1.17371 0.414968; -0.684643 -0.684643 … -0.064733 -0.186417]), Layer([1.48416; -0.0392727; … ; 0.0762837; -0.216393], [-0.775044 -0.775044 … 0.00288769 -0.0587614; -0.289933 -0.289933 … 0.0289771 0.0943521; … ; 0.124486 0.124486 … -0.0140547 0.0703901; 0.0862763 0.0862763 … 0.0219731 -0.0403008]), Layer([0.343884; -0.148704; … ; -0.331041; -0.654165], [-0.169473 -0.169473 …

In [62]:
function add_input_constraints(nnet::Network, inputs::Matrix{Float64}, m::Model)
    n_inputs = nnet.layer_sizes[1]
    @variable(m, x_in[1:n_inputs])
    @constraint(m, constr_in[i=1:n_inputs], x_in[i] == inputs[i])    
end

add_input_constraints (generic function with 1 method)

In [103]:
# test add input constraints
inputs = [1.0 2.0 3.0 4.0 5.0]
m = Model(solver = GLPKSolverLP())
add_input_constraints(large_nnet, inputs, m)

5-element Array{JuMP.ConstraintRef,1}:
 x_in[1] = 1
 x_in[2] = 2
 x_in[3] = 3
 x_in[4] = 4
 x_in[5] = 5

In [104]:
function add_output_constraints(nnet::Network, outputs::Matrix{Float64}, m::Model)
    n_layers = length(nnet.layer_sizes)
    n_outputs = nnet.layer_sizes[n_layers]
    @variable(m, x_out[1:n_outputs])
    @constraint(m, constr_out[i=1:n_outputs], x_out[i] == outputs[i])
end

add_output_constraints (generic function with 1 method)

In [105]:
# test add_output_constraints
outputs = [5.0 4.0 3.0 2.0 1.0]
m = Model(solver = GLPKSolverLP())
add_output_constraints(large_nnet, outputs, m)

5-element Array{JuMP.ConstraintRef,1}:
 x_out[1] = 5
 x_out[2] = 4
 x_out[3] = 3
 x_out[4] = 2
 x_out[5] = 1

In [114]:
function add_network_constraints(nnet::Network, m::Model)
    M = 1 # is this a suitable ("sufficiently large") constant?
    n_layers = length(nnet.layer_sizes)
    @variable(m, x_net[i=1:n_layers, j=1:nnet.layer_sizes[i]])
    @variable(m, delta[i=1:n_layers, j=1:nnet.layer_sizes[i]], Bin)
    for (l, layer) in enumerate(nnet.layers)
        
        #@constraint(m, x_n, x[i] >= ((x_n * layer.weights) + layer.biases)[i])
        #@constraint(m, x_n, x[i] <= ((x_n * layer.weights) + layer.biases)[i] + M*delta[i])
        @constraint(m, , x[i] >= 0)
        @constraint(m, x_n, x[i] <= M*(1 - delta[i]))  
    end
end

add_network_constraints (generic function with 1 method)

In [115]:
add_network_constraints(small_nnet, m)

delta[i,j] ∈ {0,1} ∀ i ∈ {1,2,3,4}, j ∈ {…}

In [ ]:
function initialize_constraints(inputs::Matrix{Float64}, outputs::Matrix{Float64},
                                network::Network, m::Model)
    add_input_constraints(inputs, m)
    #add_output_constraints(outputs, m)
    #add_network_constraints(network, m)
end

In [ ]:
# Setup our model and add our contraints
model = Model(solver = GLPKSolverLP()) # use GLPK LP as our solver
initialize_constraints(inputs, outputs, network, model)

# Solve the model
status = solve(model)

# Print out the solution
print(status)